In [435]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import torch
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision import transforms
from matplotlib import pyplot as plt
from torchvision import transforms
import torch.nn.functional as F
from torch.autograd import Variable

pilImage = transforms.ToPILImage()

from models import GlimpseSensor, GlimpseNetwork, CoreNetwork



### Load Images

In [278]:
mnist_trainset = datasets.MNIST('../../data', train=False, download=True,
                                   transform=transforms.Compose([
                                       transforms.ToTensor(),
#                                        transforms.Normalize((0.1307,), (0.3081,))
                                   ]))
data_loader_train = DataLoader(mnist_trainset, 64, shuffle=True, num_workers=8)

In [279]:
data, target = next(iter(data_loader_train))
pilImage(data[0])

### Test GlimpseSensor

In [434]:
mock_location = torch.Tensor((1, 0)).repeat(1,64).view(64, 2)
g = GlimpseSensor(retina_size=28)
a = g.foveate(data, locations=mock_location)
a = a.view(-1, 3, 1, 28, 28) # (batch, num_glimpse, channel, H, W)
pilImage(a[0][0])

### GlimpseNetwork

In [441]:
glimseNetwork = GlimpseNetwork(
                     num_hidden_glimpse = 128,
                     num_hidden_location = 128,
                     retina_size = 7,
                     num_glimpse = 3,
                     scale = 2,
                     channel = 1)
glimpse_location = glimseNetwork(data, mock_location)
glimpse_location

tensor([[0.0000, 0.0000, 0.2695,  ..., 0.5554, 0.0514, 0.0204],
        [0.0000, 0.0000, 0.2695,  ..., 0.5412, 0.0514, 0.0258],
        [0.0000, 0.0000, 0.2695,  ..., 0.5546, 0.0514, 0.0273],
        ...,
        [0.0000, 0.0000, 0.2695,  ..., 0.5923, 0.0514, 0.0646],
        [0.0000, 0.0000, 0.2695,  ..., 0.5384, 0.0514, 0.0324],
        [0.0000, 0.0000, 0.2695,  ..., 0.5763, 0.0514, 0.0098]],
       grad_fn=<ReluBackward>)

### Core Network

In [438]:
h_prev = torch.zeros((256))

In [442]:
coreNetwork = CoreNetwork(num_glimpse_location = 256, num_h=256)
c = coreNetwork(glimpse_location = glimpse_location, h_prev = h_prev)
c.shape

torch.Size([64, 256])